In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.python.keras.models import Sequential


from tensorflow.keras.datasets import cifar10
import numpy as np

In [18]:
def create_denoising_autoencoder():
    input_img = Input(shape=(32, 32, 3))

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder



In [29]:
# Cargar el dataset CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalizar los datos
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32')   / 255.

In [30]:

# Generar datos ruidosos
noise_factor = 0.10
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_train_noisy = np.clip(X_train_noisy, 0., 1.)

X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

In [21]:
autoencoder = create_denoising_autoencoder()


# Entrenar el autoencoder
autoencoder.fit(X_train_noisy, X_train, epochs=30, batch_size=256, shuffle=True, validation_data=(X_test_noisy, X_test))

Epoch 1/30
196/196 [==============================] - 59s 293ms/step - loss: 0.5910 - val_loss: 0.5721
Epoch 2/30
196/196 [==============================] - 58s 297ms/step - loss: 0.5687 - val_loss: 0.5674
Epoch 3/30
196/196 [==============================] - 54s 275ms/step - loss: 0.5657 - val_loss: 0.5654
Epoch 4/30
196/196 [==============================] - 49s 252ms/step - loss: 0.5643 - val_loss: 0.5647
Epoch 5/30
196/196 [==============================] - 52s 267ms/step - loss: 0.5633 - val_loss: 0.5640
Epoch 6/30
196/196 [==============================] - 50s 256ms/step - loss: 0.5626 - val_loss: 0.5629
Epoch 7/30
196/196 [==============================] - 51s 259ms/step - loss: 0.5620 - val_loss: 0.5630
Epoch 8/30
196/196 [==============================] - 53s 272ms/step - loss: 0.5616 - val_loss: 0.5637
Epoch 9/30
196/196 [==============================] - 50s 256ms/step - loss: 0.5612 - val_loss: 0.5618
Epoch 10/30
196/196 [==============================] - 55s 279ms/step - l

In [22]:
autoencoder.save('autoencoder.h5')

In [ ]:
autoencoder_model = tf.keras.models.load_model('autoencoder.h5')

# Mostrar las imágenes originales y las imágenes con ruido
import matplotlib.pyplot as plt

n = 4
plt.figure(figsize=(20, 4))

for i in range(n):
    # Imagen original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i])
    plt.title('Original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Imagen con ruido
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(X_test_noisy[i])
    plt.title('Con ruido')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()
